<a href="https://colab.research.google.com/github/tsato-code/colab_notebooks/blob/main/pytorch_vsumm_reinforce.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Sun Apr  4 13:48:16 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.67       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [16]:
!git clone https://github.com/Harryjun/pytorch-vsumm-reinforce.git

Cloning into 'pytorch-vsumm-reinforce'...
remote: Enumerating objects: 160, done.
remote: Total 160 (delta 0), reused 0 (delta 0), pack-reused 160
Receiving objects: 100% (160/160), 405.58 KiB | 11.59 MiB/s, done.
Resolving deltas: 100% (83/83), done.


In [62]:
# 以下のページから vsumm_datasets.zip をダウンロード
# https://drive.google.com/u/0/uc?id=1Bf0beMN_ieiM3JpprghaoOwQe9QJIyAN&export=download

!pip install gdown
%cd /content/pytorch-vsumm-reinforce
!gdown "https://drive.google.com/uc?export=download&id=1Bf0beMN_ieiM3JpprghaoOwQe9QJIyAN" -O datasets.zip
!unzip datasets.zip

/content/pytorch-vsumm-reinforce
Downloading...
From: https://drive.google.com/uc?export=download&id=1Bf0beMN_ieiM3JpprghaoOwQe9QJIyAN
To: /content/pytorch-vsumm-reinforce/datasets.zip
173MB [00:00, 282MB/s]
Archive:  datasets.zip
replace datasets/eccv16_dataset_ovp_google_pool5.h5? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace datasets/eccv16_dataset_tvsum_google_pool5.h5? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace datasets/eccv16_dataset_summe_google_pool5.h5? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace datasets/readme.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace datasets/eccv16_dataset_youtube_google_pool5.h5? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [39]:
# ディレクトリ構造
import os
for dirname, _, filenames in os.walk('./'):
    for filename in filenames:
        path = os.path.join(dirname, filename)
        if '/.' not in path:
            print(os.path.join(dirname, filename))

./datasets00.zip
./datasets01.zip
./datasets.zip
./uc?export=download&id=1Bf0beMN_ieiM3JpprghaoOwQe9QJIyAN.1
./pytorch-vsumm-reinforce/vsum_tools.py
./pytorch-vsumm-reinforce/summary2video.py
./pytorch-vsumm-reinforce/video_forward2.py
./pytorch-vsumm-reinforce/parse_json.sh
./pytorch-vsumm-reinforce/main.py
./pytorch-vsumm-reinforce/__init__.py
./pytorch-vsumm-reinforce/models.py
./pytorch-vsumm-reinforce/generate_dataset.py
./pytorch-vsumm-reinforce/video_forward.py
./pytorch-vsumm-reinforce/test_video.py
./pytorch-vsumm-reinforce/vsumm_datasets.zip
./pytorch-vsumm-reinforce/README.md
./pytorch-vsumm-reinforce/knapsack.py
./pytorch-vsumm-reinforce/rewards.py
./pytorch-vsumm-reinforce/LICENSE
./pytorch-vsumm-reinforce/parse_json.py
./pytorch-vsumm-reinforce/parse_log.py
./pytorch-vsumm-reinforce/visualize_results.py
./pytorch-vsumm-reinforce/create_split.py
./pytorch-vsumm-reinforce/utils.py
./pytorch-vsumm-reinforce/vsum_tool.py
./pytorch-vsumm-reinforce/feature_extractor.py
./pytorc

In [41]:
%cd /content/pytorch-vsumm-reinforce
!python create_split.py -d datasets/eccv16_dataset_summe_google_pool5.h5 --save-dir datasets --save-name summe_splits  --num-splits 5

/content/pytorch-vsumm-reinforce
Args:Namespace(dataset='datasets/eccv16_dataset_summe_google_pool5.h5', num_splits=5, save_dir='datasets', save_name='summe_splits', train_percent=0.8)
Goal: randomly split data for 5 times, 80.0% for training and the rest for testing
Loading dataset from datasets/eccv16_dataset_summe_google_pool5.h5
Split breakdown: # total videos 25. # train videos 20. # test videos 5
Splits saved to datasets/summe_splits.json


In [42]:
!python main.py -d datasets/eccv16_dataset_summe_google_pool5.h5 -s datasets/summe_splits.json -m summe --gpu 0 --save-dir log/summe-split0 --split-id 0 --verbose

Args:Namespace(beta=0.01, dataset='datasets/eccv16_dataset_summe_google_pool5.h5', evaluate=False, gamma=0.1, gpu='0', hidden_dim=256, input_dim=1024, lr=1e-05, max_epoch=60, metric='summe', num_episode=5, num_layers=1, resume='', rnn_cell='lstm', save_dir='log/summe-split0', save_results=False, seed=1, split='datasets/summe_splits.json', split_id=0, stepsize=30, use_cpu=False, verbose=True, weight_decay=1e-05)
Currently using GPU 0
Initialize dataset datasets/eccv16_dataset_summe_google_pool5.h5
# total videos 25. # train videos 20. # test videos 5
Initialize model
Model size: 2.62605M
==> Start training
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1709: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/content/pytorch-vsumm-reinforce/rewards.py:47: UserWarning: This overload of addmm_ is deprecated:
	addmm_(Number beta, Number alpha, Tensor mat1, Tensor mat2

In [44]:
!python main.py -d datasets/eccv16_dataset_summe_google_pool5.h5 -s datasets/summe_splits.json -m summe --gpu 0 --save-dir log/summe-split0 --split-id 0 --evaluate --resume /content/pytorch-vsumm-reinforce/log/summe-split0/model_epoch60.pth.tar --verbose --save-results

Args:Namespace(beta=0.01, dataset='datasets/eccv16_dataset_summe_google_pool5.h5', evaluate=True, gamma=0.1, gpu='0', hidden_dim=256, input_dim=1024, lr=1e-05, max_epoch=60, metric='summe', num_episode=5, num_layers=1, resume='/content/pytorch-vsumm-reinforce/log/summe-split0/model_epoch60.pth.tar', rnn_cell='lstm', save_dir='log/summe-split0', save_results=True, seed=1, split='datasets/summe_splits.json', split_id=0, stepsize=30, use_cpu=False, verbose=True, weight_decay=1e-05)
Currently using GPU 0
Initialize dataset datasets/eccv16_dataset_summe_google_pool5.h5
# total videos 25. # train videos 20. # test videos 5
Initialize model
Model size: 2.62605M
Loading checkpoint from '/content/pytorch-vsumm-reinforce/log/summe-split0/model_epoch60.pth.tar'
Evaluate only
==> Test
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1709: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid

In [46]:
!python visualize_results.py -p /content/pytorch-vsumm-reinforce/log/summe-split0/result.h5

Done video video_11. # frames 1612.
Done video video_25. # frames 3120.
Done video video_5. # frames 5131.
Done video video_6. # frames 5075.
Done video video_8. # frames 1286.


In [47]:
!python parse_log.py -p /content/pytorch-vsumm-reinforce/log/summe-split0/log_train.txt

In [49]:
!python parse_json.py -p /content/pytorch-vsumm-reinforce/log/summe-split0/rewards.json -i 0

Traceback (most recent call last):
  File "parse_json.py", line 26, in <module>
    key = keys[args.idx]
TypeError: 'dict_keys' object is not subscriptable


In [65]:
# サンプル動画を取得
%cd /content
!wget https://file-examples-com.github.io/uploads/2017/04/file_example_MP4_480_1_5MG.mp4 -O /content/test01.mp4

/content
--2021-04-04 15:49:56--  https://file-examples-com.github.io/uploads/2017/04/file_example_MP4_480_1_5MG.mp4
Resolving file-examples-com.github.io (file-examples-com.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to file-examples-com.github.io (file-examples-com.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1570024 (1.5M) [video/mp4]
Saving to: ‘test01.mp4’

test01.mp4          100%[===================>]   1.50M  --.-KB/s    in 0.07s   

2021-04-04 15:49:56 (21.5 MB/s) - ‘test01.mp4’ saved [1570024/1570024]



In [66]:
from IPython.display import HTML
from base64 import b64encode

# サンプル動画
path = '/content/test01.mp4'
mp4 = open(path,'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

In [ ]:
# ダウンロードした .h5 ファイルが、以前ダウンロードしたものと同じか確認する。
# 特徴量抽出のコードを確認してみる。
# https://github.com/SinDongHwan/pytorch-vsumm-reinforce/blob/master/utils/generate_dataset.py

In [71]:
!pip install pytube
from pytube import YouTube
YouTube("https://www.youtube.com/watch?v=rwt3oS_mFVQ").streams.first().download("/content/test02")

'/content/test02.mp4/【ホットペッパービューティー】～ナダルとオカンの押し問答\u3000美容室篇\u3000～.mp4'

In [72]:
# サンプル動画
path = '/content/test02/【ホットペッパービューティー】～ナダルとオカンの押し問答　美容室篇　～.mp4'
mp4 = open(path,'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)